In [10]:
import tensorflow as tf


In [11]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
import pandas as pd
from IPython.display import display, HTML

In [9]:
from google.colab import drive
drive.mount('/content/drive')


train_path = '/content/drive/MyDrive/sampledata/Train/ '
test_path = '/content/drive/MyDrive/sampledata/Test/ '




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

# Define the paths to your dataset
train_data_dir = '/content/drive/MyDrive/sampledata/Train'
test_data_dir = '/content/drive/MyDrive/sampledata/Test'

# Set the image dimensions and batch size
img_width, img_height = 64, 64
batch_size = 28

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=None,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False
)
test_datagen = ImageDataGenerator(
    rescale=None,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for mole classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model
# test_generator = test_datagen.flow_from_directory(
#     test_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# prediction_of_y = model.predict(test_generator)
# prediction_of_y = (prediction_of_y > 0.5).astype(int)
# class_labels = list(test_generator.class_indices.keys())
# report = classification_report(test_generator.classes, prediction_of_y, target_names=class_labels)
# print(report)
checkpoint_path = "."
model.save_weights(checkpoint_path)

Found 853 images belonging to 2 classes.
Found 118 images belonging to 2 classes.
Epoch 1/5
30/30 [==============================] - 89s 3s/step - loss: 0.0264 - accuracy: 0.9939 - val_loss: 8.7594 - val_accuracy: 0.8661
Epoch 2/5
30/30 [==============================] - 75s 2s/step - loss: 9.5461e-14 - accuracy: 1.0000 - val_loss: 9.9064 - val_accuracy: 0.8571
Epoch 3/5
30/30 [==============================] - 79s 3s/step - loss: 7.4670e-13 - accuracy: 1.0000 - val_loss: 9.9283 - val_accuracy: 0.8571
Epoch 4/5
30/30 [==============================] - 79s 3s/step - loss: 2.6740e-12 - accuracy: 1.0000 - val_loss: 9.2821 - val_accuracy: 0.8661
Epoch 5/5
30/30 [==============================] - 79s 3s/step - loss: 8.9520e-13 - accuracy: 1.0000 - val_loss: 9.3163 - val_accuracy: 0.8661


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
model.save_weights("/contents/checkpoint1.ckpt")

In [43]:
model.load_weights("/contents/checkpoint1.ckpt")
import cv2
# Define the paths to your dataset
train_data_dir = '/content/drive/MyDrive/sampledata/Train'
test_data_dir = '/content/drive/MyDrive/sampledata/Test'

# Set the image dimensions and batch size
img_width, img_height = 64, 64
batch_size = 28

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=None,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False
)
test_datagen = ImageDataGenerator(
    rescale=None,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

prediction_of_y = model.predict(test_generator)
prediction_of_y = (prediction_of_y > 0.5).astype(int)
class_labels = list(test_generator.class_indices.keys())
report = classification_report(test_generator.classes, prediction_of_y, target_names=class_labels)
print(report)


loss, acc = model.evaluate(validation_generator, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

Found 853 images belonging to 2 classes.
Found 118 images belonging to 2 classes.
Found 118 images belonging to 2 classes.
5/5 [==============================] - 16s 3s/step
                            precision    recall  f1-score   support

                not benign       0.86      1.00      0.93       102
pigmented benign keratosis       0.00      0.00      0.00        16

                  accuracy                           0.86       118
                 macro avg       0.43      0.50      0.46       118
              weighted avg       0.75      0.86      0.80       118



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Restored model, accuracy: 86.44%


In [8]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# Load the training and testing data
train_data = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_dataset = train_data.flow_from_directory(
    '/content/drive/MyDrive/sampledata/Test',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

test_data = ImageDataGenerator(rescale=1./255)
test_dataset = test_data.flow_from_directory(
    '/content/drive/MyDrive/sampledata/Train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=5)

# Evaluate the model
model.evaluate(test_dataset)

# Make predictions
predictions = model.predict(test_dataset)

# Print the predictions
for i in range(len(predictions)):
    print('Prediction:', predictions[i], 'Actual:', test_dataset.classes[i])


Found 118 images belonging to 2 classes.
Found 853 images belonging to 2 classes.


NameError: ignored